# QA: Plot behavioral model comparisons
Natalia Vélez, February 2023

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_context('talk')
sns.set_style('white')

## Load data

Teacher data

In [ ]:
teacher_df = pd.read_csv('../2_behavioral/outputs/teaching_behavior.csv')
teacher_df['count'] = 1-pd.isna(teacher_df['example'])*1
teacher_df['num_hint'] = teacher_df.groupby(['subject', 'run', 'problem'])['count'].agg('cumsum')
teacher_df = teacher_df.dropna(subset=['example', 'rating'])
teacher_df.head()

Student data

In [ ]:
student_df = pd.read_csv('outputs/student_beliefs.csv').sort_values(by=['worker', 'num_trial'])
student_df.head()

Model predictions

In [ ]:
model_df = pd.read_csv('../2_behavioral/outputs/model_regressor_values.csv')
model_df['count'] = 1
model_df['num_hint'] = model_df.groupby(['subject', 'run', 'problem'])['count'].agg('cumsum')
model_df.head()

Regression model comparisons:

In [ ]:
teacher_vs_time = pd.read_csv('outputs/teacher_vs_time.csv')
teacher_vs_time['term'] = (
    teacher_vs_time['term'].astype('category')
    .cat.reorder_categories(['time_teacher', 'belief_teacher', 'full_teacher'])
    .cat.rename_categories({'time_teacher': 'Time', 'belief_teacher': 'Beliefs', 'full_teacher': 'Time +\nBeliefs'})
)

teacher_vs_time.head()

In [ ]:
student_vs_time = pd.read_csv('outputs/student_vs_time.csv')
student_vs_time['term'] = (
    student_vs_time['term'].astype('category')
    .cat.reorder_categories(['time_student', 'belief_student', 'full_student'])
    .cat.rename_categories({'time_student': 'Time', 'belief_student': 'Beliefs', 'full_student': 'Time +\nBeliefs'})
)

student_vs_time.head()

## Check 1: Correlations between belief regressors & time

Model-based belief regressors:

In [ ]:
model_time_corr = model_df[['num_hint', 'pTrue_unscaled']].corr('spearman')
print(stats.spearmanr(model_df['num_hint'], model_df['pTrue_unscaled']))
mt_r = model_time_corr.loc['num_hint', 'pTrue_unscaled']

g = sns.jointplot(data=model_df, x='num_hint', y='pTrue_unscaled', kind='hex')
g.ax_joint.set(xlabel = '# Examples', ylabel = r'Belief in correct answer', xticks=[1,2,3], yticks=[0,.25,.5,.75,1])
g.ax_joint.text(0.95,0.95, r"$r = %.2f$" % (mt_r))
g.fig.suptitle('GLM 1: Model-based', y = 1.025)

plt.savefig('plots/model_vs_time_corr.pdf', bbox_inches='tight')

Empirical belief regressors:

In [ ]:
empirical_time_corr = student_df[['num_hint', 'belief_in_true']].corr('spearman')
print(stats.spearmanr(student_df['num_hint'], student_df['belief_in_true']))
et_r = empirical_time_corr.loc['num_hint', 'belief_in_true']

g = sns.jointplot(data=student_df, x='num_hint', y='belief_in_true', kind='hex')
g.ax_joint.set(xlabel = '# Examples', ylabel = r'Belief in correct answer', xticks=[1,2,3], yticks=[0,.25,.5,.75,1])
g.ax_joint.text(0.95,0.95, r"$r = %.2f$" % (et_r))
g.fig.suptitle('GLM 2: Human learners', y=1.025)

plt.savefig('plots/empirical_vs_time_corr.pdf', bbox_inches='tight')

## Check 2: Teacher ratings

Plot change in ratings over time:

In [ ]:
problem_pal = ['#333']*teacher_df.problem.nunique()
g = sns.relplot(data=teacher_df, x='num_hint', y='rating', hue='problem', kind='line', height=4.5, aspect=1,legend=False, palette=problem_pal, alpha=.01, ci=False)
g.set(xlabel='# Examples', ylabel='Rating')
sns.lineplot(data=teacher_df, x='num_hint', y='rating', ax=g.axes[0,0], ci=False, linewidth=6, color='#00A2FF')

plt.savefig('plots/teacher_ratings_over_time.pdf', bbox_inches='tight')

Plot model comparison:

In [ ]:
# color palette for exposition
pal = ['#aaa', '#aaa', '#333']

# draw plot
fig,ax = plt.subplots(figsize=(4.5,3))
sns.barplot(data=teacher_vs_time, x='term', y='BIC', palette=pal)
ax.set(xlabel='', title='Teacher ratings')

plt.savefig('plots/teacher_rating_model_comparison.pdf', bbox_inches='tight')

## Check 3: Student performance

Plot change in performance over time:

In [ ]:
problem_pal = ['#333']*student_df.problem.nunique()
g = sns.relplot(data=student_df, x='num_hint', y='belief_in_true', hue='problem', kind='line', height=4.5, aspect=1,legend=False, palette=problem_pal, alpha=.01, ci=False)
g.set(xlabel='# Examples', ylabel='Belief in true hypothesis')
sns.lineplot(data=student_df, x='num_hint', y='belief_in_true', ax=g.axes[0,0], ci=False, linewidth=6, color='#00A2FF')

plt.savefig('plots/student_performance_over_time.pdf', bbox_inches='tight')

Plot model comparison:

In [ ]:
fig,ax = plt.subplots(figsize=(4.5,3))
sns.barplot(data=student_vs_time, x='term', y='BIC', palette=pal)
ax.set(xlabel='', title='Student performance')

plt.savefig('plots/student_performance_model_comparison.pdf', bbox_inches='tight')